# EventKG - Retrieving events for experiments

The aim of the notebook is to automatically retrieve events for the search experiment, with other information like start and end dates.

Before running the notebook, ensure to have the followings:
* EventKG downloaded and preprocessed, cf. `eventkg-filtering.ipynb`
* Subset of EventKG loaded in [GraphDB](https://graphdb.ontotext.com)
* GraphDB endpoint active (Repositories name `eventkg`)

In [5]:
import io
import os
import json
import yaml
import requests
import psutil
import wandb
from tqdm import tqdm

import ray
from ray.util.multiprocessing.pool import Pool
import pandas as pd
from settings import FOLDER_PATH, WANDB_USER#

In [6]:
HEADERS = {
    "Accept": "text/csv"
}

ENDPOINT = "http://localhost:7200/repositories/eventkg"

NB_EVENTS = 1333

# Folder where data necessary to run experiments will be saved
# This folder should contain the following sub folders: `config`, `gs_events` and `referents`
FOLDER_SAVE_DATA = os.path.join(FOLDER_PATH, "data-test")

NB_CPUS = psutil.cpu_count(logical=False)

## 1. Retrieving events with the most sub events

Using SPARQL Query + GraphDB endpoint

In [4]:
QUERY_RETRIEVE_EVENTS = """
PREFIX sem: <http://semanticweb.cs.vu.nl/2009/11/sem/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
SELECT ?eventKG (COUNT(DISTINCT ?subEventKG) as ?nbSubEvent)
WHERE {
    
 ?event sem:hasSubEvent* ?subEvent .
 ?event sem:hasBeginTimeStamp ?startTimeEvent .
 ?event sem:hasEndTimeStamp ?endTimeEvent .
 ?event owl:sameAs ?eventKG .

 ?subEvent owl:sameAs ?subEventKG .
 ?subEvent sem:hasBeginTimeStamp ?startTimeSubEvent .
 ?subEvent sem:hasEndTimeStamp ?endTimeSubEvent .
 
 FILTER (?endTimeSubEvent >= ?startTimeEvent) .
 FILTER (?startTimeSubEvent <= ?endTimeEvent) .
 FILTER( strStarts( str(?eventKG), "http://dbpedia" ) ) .
 FILTER( strStarts( str(?subEventKG), "http://dbpedia" ) ) .
}
GROUP BY ?eventKG
ORDER BY DESC(?nbSubEvent)
"""

QUERY_RETRIEVE_EVENTS = \
    f"{QUERY_RETRIEVE_EVENTS}\nLIMIT {NB_EVENTS}"


In [5]:
response = requests.get(ENDPOINT, headers=HEADERS,
                        params={"query": QUERY_RETRIEVE_EVENTS})

In [9]:
df_events = pd.read_csv(
    io.StringIO(response.content.decode('utf-8'))
)
df_events

,eventKG,nbSubEvent
0,http://dbpedia.org/resource/World_War_II,1437
1,http://dbpedia.org/resource/Coalition_Wars,996
2,http://dbpedia.org/resource/Cold_War,883
3,http://dbpedia.org/resource/World_War_I,665
4,http://dbpedia.org/resource/American_Civil_War,630
...,...,...
1328,http://dbpedia.org/resource/Weightlifting_at_t...,11
1329,http://dbpedia.org/resource/Weightlifting_at_t...,11
1330,http://dbpedia.org/resource/Weightlifting_at_t...,11
1331,http://dbpedia.org/resource/Western_Allied_inv...,11


## 2. Retrieving info for each selected event

* Ground truth events from EventKG 
* Referents (URI mapping)
* Start/End dates


### 2.1. Ground truth for each event

Ground truth = event part of that event in EventKG

In [10]:
df_events.to_csv("df_events.csv")

In [14]:
df_events = pd.read_csv("df_events.csv")

In [12]:
QUERY_GROUND_TRUTH_TEMPLATE = """
PREFIX sem: <http://semanticweb.cs.vu.nl/2009/11/sem/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
SELECT DISTINCT(?subEventKG as ?linkDBpediaEn)
WHERE {
    
?event owl:sameAs <event-to-replace> .
?event sem:hasSubEvent* ?subEvent .
?subEvent owl:sameAs ?subEventKG .
    
?event sem:hasBeginTimeStamp ?startTimeEvent .
?event sem:hasEndTimeStamp ?endTimeEvent .

?subEvent sem:hasBeginTimeStamp ?startTimeSubEvent .
?subEvent sem:hasEndTimeStamp ?endTimeSubEvent .
    
FILTER( strStarts( str(?subEventKG), "http://dbpedia" ) ) .
FILTER (?endTimeSubEvent >= ?startTimeEvent) .
FILTER (?startTimeSubEvent <= ?endTimeEvent) .
}
"""

In [13]:
events = df_events.eventKG.values
for i in tqdm(range(len(events))):
    event = events[i]
    name = event.split("/")[-1]
    query = QUERY_GROUND_TRUTH_TEMPLATE.replace(
        "event-to-replace", event
    )
    response = requests.get(ENDPOINT, headers=HEADERS,
                            params={"query": query})
    pd.read_csv(io.StringIO(response.content.decode('utf-8'))) \
        .to_csv(os.path.join(FOLDER_SAVE_DATA,
                             "gs_events",
                             f"{name}.csv"))

100%|██████████| 1333/1333 [00:32<00:00, 40.87it/s]


### 2.2. URI referents for each sub event

Due to differences in dataset version, URIs can vary over time, the aim of this section is to retrieve a unique ID referent for each set of URIs.


In [7]:
from src.get_equivalent_url import get_equivalent_url

In [8]:
def add_equivalent_url(df_path, save_path):
    if not os.path.exists(save_path):
        get_equivalent_url(df_path=df_path, save_path=save_path)

In [9]:
#get_equivalent_url('/Users/ines/Projects/graph_search_framework/data-test/gs_events/2004_Summer_Olympics.csv',
#                   '/Users/ines/Projects/graph_search_framework/data-test/referents/2004_Summer_Olympics.json')

In [10]:
csv_folder = os.path.join(FOLDER_SAVE_DATA, "gs_events")
json_folder = os.path.join(FOLDER_SAVE_DATA, "referents")

csv_files = os.listdir(csv_folder)

args = [
    (os.path.join(csv_folder, csv_file),
     os.path.join(
         json_folder,
         f"{os.path.splitext(csv_file)[0]}.json")) \
             for csv_file in csv_files
]

pool = Pool(processes=NB_CPUS)
pool.starmap(add_equivalent_url, args)


100%|█████████▉| 302/303 [00:59<00:00,  5.53it/s]


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

100%|██████████| 303/303 [00:59<00:00,  5.07it/s]


### 2.3. Start and End dates of each event

Minimum start date among all start dates, maximum end date among all end dates.

Start date must be before end date.

In [26]:
QUERY_DATES_TEMPLATE = """
PREFIX sem: <http://semanticweb.cs.vu.nl/2009/11/sem/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
SELECT (min(?startTimeEvent) as ?min) (max(?endTimeEvent) as ?max)
WHERE {
    
 ?event owl:sameAs <event-to-replace> .
 ?event sem:hasSubEvent* ?subEvent .
 ?event sem:hasBeginTimeStamp ?startTimeEvent .
 OPTIONAL { ?event sem:hasEndTimeStamp ?endTimeEvent . }
 ?event owl:sameAs ?eventKG .

 FILTER( strStarts( str(?eventKG), "http://dbpedia" ) ) .
}
GROUP BY ?eventKG
HAVING (max(?endTimeEvent) > min(?startTimeEvent))
"""

In [28]:
query = QUERY_DATES_TEMPLATE.replace(
        "event-to-replace", "http://dbpedia.org/resource/2020_United_States_House_of_Representatives_elections")
response = requests.get(ENDPOINT, headers=HEADERS,
                    params={"query": query})
pd.read_csv(io.StringIO(response.content.decode('utf-8')))

,min,max


In [15]:
def get_dates(event):
    query = QUERY_DATES_TEMPLATE.replace(
        "event-to-replace", event)
    response = requests.get(ENDPOINT, headers=HEADERS,
                        params={"query": query})
    return pd.read_csv(io.StringIO(response.content.decode('utf-8')))

In [16]:
if ray.is_initialized() == True:
    ray.shutdown()
pool = Pool(processes=NB_CPUS)
result = pool.map(get_dates, df_events.eventKG.values)
if ray.is_initialized() == True:
    ray.shutdown()

2022-05-11 11:31:56,554	INFO pool.py:517 -- Starting local ray cluster


In [20]:
result[0]

,min,max
0,1939-09-01,1945-09-02


In [31]:
from src.triply_interface import TriplInterface

interface = TriplInterface()

def retrieve_date_triply(node):
    predicate = "http://dbpedia.org/ontology/startDate"
    triples = interface.run_request(params=dict(subject=node, predicate=predicate), filter_pred=[], filter_keep=False)
    if len(triples) == 0:
        return None, None
    elif len(triples) == 1:
        return str(triples[0][2]), str(triples[0][2])
    else:
        dates = [str(elt[2]) for elt in triples]
        return min(dates), max(dates)

In [33]:
manual_dates = {
    "2014_United_States_elections": {"start": "2014-11-04", "end": "2014-11-04"},
    "2018_United_States_elections": {"start": "2018-11-06", "end": "2018-11-06"},
    "Arab–Israeli_conflict": {"start": "1948-05-15", "end": "2021-12-31"},
    "War_on_terror": {"start": "2011-09-11", "end": "2021-12-31"},
    "Iraqi_conflict_(2003–present)": {"start": "2003-03-20", "end": "2021-12-31"},
}

In [34]:
def store_changing_config(dico):
    counter = 0
    for i, event in enumerate(df_events.eventKG.values):
        curr_df = result[i]
        name = event.split("/")[-1]
        start, end = None, None
        if curr_df.shape[0] != 0:
            start = curr_df["min"].values[0]
            end = curr_df["max"].values[0]
        else:
            start, end = retrieve_date_triply(node=event)
            if not (start and end) and name in manual_dates:
                start, end = manual_dates.get(name).get("start"), manual_dates.get(name).get("end")
        if start and end:
            dico[event] = {
                "start": event,
                "start_date": start,
                "end_date": end,
                "gold_standard": os.path.join(FOLDER_SAVE_DATA, "gs_events", f"{name}.csv"),
                "referents": os.path.join(FOLDER_SAVE_DATA, "referents", f"{name}.json"),
                "name_exp": name,
            }
        else:
            counter += 1
            print(f"Dates for {name} could not be found")
    print(f"{counter} events could not be processed further")
    return dico

dico_config = store_changing_config(dico={})

0 events could not be processed further


## 3. Prepare configuration files for sweep for each event

"predicate_filter": ["http://dbpedia.org/ontology/wikiPageWikiLink",
                         "http://dbpedia.org/ontology/wikiPageRedirects",
                         "http://dbpedia.org/ontology/wikiPageDisambiguates",
                         "http://www.w3.org/2000/01/rdf-schema#seeAlso",
                         "http://xmlns.com/foaf/0.1/depiction",
                         "http://xmlns.com/foaf/0.1/isPrimaryTopicOf",
                         "http://dbpedia.org/ontology/thumbnail",
                         "http://dbpedia.org/ontology/wikiPageExternalLink",
                         "http://dbpedia.org/ontology/wikiPageID",
                         "http://dbpedia.org/ontology/wikiPageLength",
                         "http://dbpedia.org/ontology/wikiPageRevisionID",
                         "http://dbpedia.org/property/wikiPageUsesTemplate",
                         "http://www.w3.org/2002/07/owl#sameAs",
                         "http://www.w3.org/ns/prov#wasDerivedFrom",
                         "http://dbpedia.org/ontology/wikiPageWikiLinkText",
                         "http://dbpedia.org/ontology/wikiPageOutDegree",
                         "http://dbpedia.org/ontology/abstract",
                         "http://www.w3.org/2000/01/rdf-schema#comment",
                         "http://www.w3.org/2000/01/rdf-schema#label"],

In [35]:
BASE_CONFIG = {
    "rdf_type": {
        "event": "http://dbpedia.org/ontology/Event"
    },
    "start": "http://dbpedia.org/resource/WTA_Tier_II_tournaments",
    "start_date": "1990-01-01",
    "end_date": "2008-12-31",
    "iterations": 5,
    "type_ranking": "pred_object_freq",
    "type_interface": "hdt",
    "gold_standard": "./data/gs_events/events_WTA_Tier_II_tournaments.csv",
    "referents": "./data/referents/referents_WTA_Tier_II_tournaments.json",
    "type_metrics": ["precision", "recall", "f1"],
    "filtering": {
        "what": 1,
        "where": 1,
    },
    "name_exp": "wta_tier_ii_tournament"
}

In [37]:
for event in df_events.eventKG.values:
    name = event.split("/")[-1]
    BASE_CONFIG.update(dico_config[event])
    with open(os.path.join(FOLDER_SAVE_DATA, "config", f"{name}.json"), "w", encoding='utf-8') as openfile:
        json.dump(BASE_CONFIG, openfile, indent=4)

## 4. Run weights & biases sweep for each event

In [3]:
with open(os.path.join(FOLDER_PATH, "graph_search_sweep.yaml")) as file:
    WANDB_SWEEP_CONFIG = yaml.load(file, Loader=yaml.FullLoader)

In [7]:
WANDB_SWEEP_CONFIG["parameters"]["json"]["value"] = \
    os.path.join(FOLDER_PATH, "config.json")
WANDB_SWEEP_CONFIG["name"] = "French_Revolution"

sweep_id = wandb.sweep(WANDB_SWEEP_CONFIG, project=WANDB_SWEEP_CONFIG["project"])

Create sweep with ID: bg6vyi8i
Sweep URL: https://wandb.ai/ines-blin/event-graph-search-framework/sweeps/bg6vyi8i


In [8]:
agent = f"{WANDB_USER}/{WANDB_SWEEP_CONFIG['project']}/{sweep_id}"
wandb.agent(agent)

wandb: Agent Starting Run: twiys86o with config:
	filtering_what: 1
	filtering_when: 0
	filtering_where: 0
	json: /Users/ines/Projects/graph_search_framework/config.json
	ordering_domain_range: 0
	type_ranking: pred_object_freq
wandb: Agent Starting Run: f5ca58bu with config:
	filtering_what: 1
	filtering_when: 0
	filtering_where: 0
	json: /Users/ines/Projects/graph_search_framework/config.json
	ordering_domain_range: 0
	type_ranking: entropy_pred_object_freq
wandb: Agent Starting Run: 1e3c6c6q with config:
	filtering_what: 1
	filtering_when: 0
	filtering_where: 0
	json: /Users/ines/Projects/graph_search_framework/config.json
	ordering_domain_range: 1
	type_ranking: pred_object_freq


wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: Tracking run with wandb version 0.12.15
wandb: W&B syncing is set to `offline` in this directory.  
wandb: Run `wandb online` or set WANDB_MODE=online to enable cloud syncing.


Processing node 1/1	http://dbpedia.org/resource/French_Revolution


100% 88/88 [00:00<00:00, 184.77it/s]
100% 6/6 [00:00<00:00, 56.34it/s]
/Users/ines/Projects/graph_search_framework/src/filtering.py:94: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  date_df.object = date_df.object.astype(str)
/Users/ines/Projects/graph_search_framework/src/framework.py:339: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subgraph_ingoing["iteration"] = iteration
100% 18/18 [00:00<00:00, 554.48it/s]
100% 3/3 [00:00<00:00, 631.96it/s]
wandb: WARNING Ignored wandb.init() arg project when running

Processing node 1/12	http://dbpedia.org/resource/10_August_(French_Revolution)
Processing node 2/12	http://dbpedia.org/resource/Battle_of_Kaiserslautern


...

0it [00:00, ?it/s]
100% 18/18 [00:00<00:00, 147.57it/s]
/Users/ines/Projects/graph_search_framework/src/filtering.py:94: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Processing node 11/12	http://dbpedia.org/resource/Storming_of_the_Bastille


100% 8/8 [00:00<00:00, 150.68it/s]
100% 14/14 [00:00<00:00, 141.19it/s]
/Users/ines/Projects/graph_search_framework/src/filtering.py:94: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Processing node 12/12	http://dbpedia.org/resource/Thermidorian_Reaction


100% 8/8 [00:00<00:00, 167.59it/s]
100% 30/30 [00:00<00:00, 167.09it/s]
/Users/ines/Projects/graph_search_framework/src/filtering.py:94: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

100% 1/1 [00:00<00:00, 122.63it/s]
100% 3/3 [00:00<00:00, 158.60it/s]
100% 2/2 [00:00<00:00, 487.17it/s]
100% 2/2 [00:00<00:00, 2927.96it/s]
0it [00:00, ?it/s]
100% 2/2 [00:00<00:00, 2958.94it/s]
0it [00:00, ?it/s]
100% 3/3 [00:00<00:00, 3043.03it/s]
100% 1/1 [00:00<00:00, 2704.26it/s]
100% 6/6 [00:00<00:00, 557.26it/s]
0it [00:00, ?it/s]
100% 2/2 [00:00<00:00, 3053.73it/s]
0it [00:00, ?it/s]
100% 2/2 [00:00<00:00, 3072.75it/s]
0it [00:00, ?it/s]
100% 1/1 [00:00<00:00, 2849.39it/s]
0it [00:00, ?it/s]
100% 2/2 [00:00<00:00, 2971.52it/s]
0it [00:00, ?it/s]
100% 3/3 [00

In [81]:
api = wandb.Api()
WANDB_USER = "ines-blin"
agent = f"{WANDB_USER}/{WANDB_SWEEP_CONFIG['project']}/{sweep_id}"
sweep = api.sweep(agent)

In [88]:
wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [96]:
list(sweep.runs)[0].summary

{'f1': 0.04856512141280353, 'name': 'french_revolution_entropy_pred_object_freq_domain_range_what_where_when', '_step': 5, '_timestamp': 1650641940, 'subgraph_nb_event': 13, 'subgraph_nb_event_unique': 12, '_wandb': {'runtime': 7}, 'recall': 0.024943310657596373, 'best_f1': 0.04856512141280353, '_runtime': 6, 'precision': 0.9166666666666666}